In [1]:
from google.colab import drive
import os
drive.mount('/gdrive')
os.chdir("/gdrive/My Drive/MLcontent/")

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [2]:
from pandas import read_csv
from pandas import datetime
from pandas import to_numeric
from pandas import concat
import numpy as np

%tensorflow_version 2.x
import tensorflow as tf

#tf.enable_v2_behavior()
from tensorflow.compat.v1.keras import backend as K
from keras.layers import SimpleRNN, Dense, LSTM, Bidirectional, GRU
from keras.models import Sequential
from matplotlib import pyplot as plt
from keras.callbacks import ModelCheckpoint

import os
import random as rn

import itertools
#from keras import backend as K

print(tf.__version__)

2.3.0


In [3]:
#Setting a seed for the computer's pseudorandom number generator. 
#This allows us to reproduce the results from our script:
#固定seed
n = 5
np.random.seed(100 * n)
rn.seed(10000 * n)


In [4]:
#Depending on the actual running environment, you may specify if using GPU, or only CPU.
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

#Tensorflow session configuration.
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1,
                              inter_op_parallelism_threads=1)

tf.compat.v1.set_random_seed(1000 * n)

sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
K.set_session(sess)

In [5]:
series = read_csv('Wastewater_Data.csv', header=0, parse_dates=[0], index_col=0,squeeze=True)
series = series.replace('^\s*$', np.nan, regex=True)
series = series.fillna(method='ffill')
series
#series = series.apply(to_numeric)

,Total Solids,SS,BOD5,NH3,Org-N,P-TOT,SO4,TKN,PRCP_NOOA
Date,,,,,,,,,
2001-01-01,984.0,170.0,173,15.93,14.0,4.58,72,29.97,0.00
2001-01-02,924.0,122.0,174,15.52,14.4,4.52,72,29.96,0.00
2001-01-03,1135.0,300.0,303,14.19,21.9,6.06,72,36.11,0.00
2001-01-04,1013.0,192.0,228,15.43,19.4,5.33,72,34.83,0.00
2001-01-05,1088.0,226.0,210,14.77,19.3,5.41,72,34.06,0.00
...,...,...,...,...,...,...,...,...,...
2018-12-27,892.0,264.0,157,12.45,11.1,3.84,85,23.53,0.66
2018-12-28,776.0,104.0,94,10.61,6.2,1.97,85,16.76,0.08
2018-12-29,898.0,148.0,83,11.38,7.4,2.7,85,18.81,0.05


In [6]:
#narmalization
#lag defines how many historical data are used to predict a specific wasterwater characteristic (e.g., BOD5)
lag = 7

#num_features defines how many historical wasterwater characteristics are used. We have 9 in total (i.e., TS, BOD5, NH3, etc.)
num_features = 9
target_dict = {}
for i, j in enumerate(series.columns):
    target_dict[j] = i
from pandas import DataFrame
from sklearn import preprocessing
names = series.columns
x = series.values #returns a numpy array
std_scaler = preprocessing.StandardScaler()
x_scaled = std_scaler.fit(x)
meandata = list(x_scaled.mean_)
stddata = list(x_scaled.scale_)
x_scaled = std_scaler.transform(x)
series_normalized = DataFrame(x_scaled, columns=names) 
#The following line of code will do the same normalization as the code above.
#series_normalized = (series - np.min(series))/(np.max(series)-np.min(series))
series_normalized

,Total Solids,SS,BOD5,NH3,Org-N,P-TOT,SO4,TKN,PRCP_NOOA
0,0.123883,-0.103696,0.285108,0.175839,0.570441,0.244988,-0.800344,0.444726,-0.333629
1,-0.148157,-0.462668,0.299175,0.096102,0.652383,0.207930,-0.800344,0.443520,-0.333629
2,0.808515,0.868520,2.113788,-0.162557,2.188784,1.159068,-0.800344,1.185309,-0.333629
3,0.255368,0.060833,1.058780,0.078599,1.676650,0.708204,-0.800344,1.030920,-0.333629
4,0.595418,0.315105,0.805578,-0.049758,1.656165,0.757614,-0.800344,0.938046,-0.333629
...,...,...,...,...,...,...,...,...,...
6569,-0.293244,0.599291,0.060039,-0.500953,-0.023634,-0.212053,-0.011446,-0.332042,1.726609
6570,-0.819187,-0.597282,-0.826167,-0.858796,-1.027417,-1.367006,-0.011446,-1.148613,-0.083904
6571,-0.266041,-0.268225,-0.980901,-0.709046,-0.781592,-0.916142,-0.011446,-0.901350,-0.177551
6572,-0.356720,-0.447711,-0.741766,-0.483449,-0.986446,-1.070547,-0.011446,-0.888082,-0.333629


In [7]:
# table2lags() Shifts a dataFrame along its time axis (i.e., index) n steps 
# (moving down/up if step is a positive/negative number), determined by min_lag and max_lag, 
# and merge all shiffted dataframes into a single one and return, 
# without including the original DataFrame (the one shifted 0 step).
# "values" is a list, and each item in "values" is a shifted dataframe.
# Input: Table: a DataFrame; max_lag: the maximum shifting; min_lag: the minimum shifting; 
# In this module, no need to include the original dataframe or shift the dataframe up, and thus min_lag = 1 by default
# separator: used to concatenate the step value (e.g., 1, 2, 3) to the original column label. E.g., SO4_1, SO4_2, etc.
# Output: a dataframe
def table2lags(table, max_lag, min_lag=1, separator='_'):
    values = []
    for i in range(min_lag, max_lag + 1):
        #append shiffted dataframe into the list (i.e., values)
        values.append(table.shift(i).copy()) 
        #replace the last item or dataframe's columns by column_n; n is the shift step  
        values[-1].columns = [c + separator + str(i) for c in table.columns] 
    #pandas.concat is used to merge all dataframes (as items in values) into a single dataframe
    return concat(values, axis=1) 

#prepare all historical (e.g., one day ago, two day ago,...) data into one dataframe. 
#For all missing data, replaced by its closest future values

X = table2lags(series_normalized, lag)
X = X.fillna(method='bfill')
X

,Total Solids_1,SS_1,BOD5_1,NH3_1,Org-N_1,P-TOT_1,SO4_1,TKN_1,PRCP_NOOA_1,Total Solids_2,SS_2,BOD5_2,NH3_2,Org-N_2,P-TOT_2,SO4_2,TKN_2,PRCP_NOOA_2,Total Solids_3,SS_3,BOD5_3,NH3_3,Org-N_3,P-TOT_3,SO4_3,TKN_3,PRCP_NOOA_3,Total Solids_4,SS_4,BOD5_4,NH3_4,Org-N_4,P-TOT_4,SO4_4,TKN_4,PRCP_NOOA_4,Total Solids_5,SS_5,BOD5_5,NH3_5,Org-N_5,P-TOT_5,SO4_5,TKN_5,PRCP_NOOA_5,Total Solids_6,SS_6,BOD5_6,NH3_6,Org-N_6,P-TOT_6,SO4_6,TKN_6,PRCP_NOOA_6,Total Solids_7,SS_7,BOD5_7,NH3_7,Org-N_7,P-TOT_7,SO4_7,TKN_7,PRCP_NOOA_7
0,0.123883,-0.103696,0.285108,0.175839,0.570441,0.244988,-0.800344,0.444726,-0.333629,0.123883,-0.103696,0.285108,0.175839,0.570441,0.244988,-0.800344,0.444726,-0.333629,0.123883,-0.103696,0.285108,0.175839,0.570441,0.244988,-0.800344,0.444726,-0.333629,0.123883,-0.103696,0.285108,0.175839,0.570441,0.244988,-0.800344,0.444726,-0.333629,0.123883,-0.103696,0.285108,0.175839,0.570441,0.244988,-0.800344,0.444726,-0.333629,0.123883,-0.103696,0.285108,0.175839,0.570441,0.244988,-0.800344,0.444726,-0.333629,0.123883,-0.103696,0.285108,0.175839,0.570441,0.244988,-0.800344,0.444726,-0.333629
1,0.123883,-0.103696,0.285108,0.175839,0.570441,0.244988,-0.800344,0.444726,-0.333629,0.123883,-0.103696,0.285108,0.175839,0.570441,0.244988,-0.800344,0.444726,-0.333629,0.123883,-0.103696,0.285108,0.175839,0.570441,0.244988,-0.800344,0.444726,-0.333629,0.123883,-0.103696,0.285108,0.175839,0.570441,0.244988,-0.800344,0.444726,-0.333629,0.123883,-0.103696,0.285108,0.175839,0.570441,0.244988,-0.800344,0.444726,-0.333629,0.123883,-0.103696,0.285108,0.175839,0.570441,0.244988,-0.800344,0.444726,-0.333629,0.123883,-0.103696,0.285108,0.175839,0.570441,0.244988,-0.800344,0.444726,-0.333629
2,-0.148157,-0.462668,0.299175,0.096102,0.652383,0.207930,-0.800344,0.443520,-0.333629,0.123883,-0.103696,0.285108,0.175839,0.570441,0.244988,-0.800344,0.444726,-0.333629,0.123883,-0.103696,0.285108,0.175839,0.570441,0.244988,-0.800344,0.444726,-0.333629,0.123883,-0.103696,0.285108,0.175839,0.570441,0.244988,-0.800344,0.444726,-0.333629,0.123883,-0.103696,0.285108,0.175839,0.570441,0.244988,-0.800344,0.444726,-0.333629,0.123883,-0.103696,0.285108,0.175839,0.570441,0.244988,-0.800344,0.444726,-0.333629,0.123883,-0.103696,0.285108,0.175839,0.570441,0.244988,-0.800344,0.444726,-0.333629
3,0.808515,0.868520,2.113788,-0.162557,2.188784,1.159068,-0.800344,1.185309,-0.333629,-0.148157,-0.462668,0.299175,0.096102,0.652383,0.207930,-0.800344,0.443520,-0.333629,0.123883,-0.103696,0.285108,0.175839,0.570441,0.244988,-0.800344,0.444726,-0.333629,0.123883,-0.103696,0.285108,0.175839,0.570441,0.244988,-0.800344,0.444726,-0.333629,0.123883,-0.103696,0.285108,0.175839,0.570441,0.244988,-0.800344,0.444726,-0.333629,0.123883,-0.103696,0.285108,0.175839,0.570441,0.244988,-0.800344,0.444726,-0.333629,0.123883,-0.103696,0.285108,0.175839,0.570441,0.244988,-0.800344,0.444726,-0.333629
4,0.255368,0.060833,1.058780,0.078599,1.676650,0.708204,-0.800344,1.030920,-0.333629,0.808515,0.868520,2.113788,-0.162557,2.188784,1.159068,-0.800344,1.185309,-0.333629,-0.148157,-0.462668,0.299175,0.096102,0.652383,0.207930,-0.800344,0.443520,-0.333629,0.123883,-0.103696,0.285108,0.175839,0.570441,0.244988,-0.800344,0.444726,-0.333629,0.123883,-0.103696,0.285108,0.175839,0.570441,0.244988,-0.800344,0.444726,-0.333629,0.123883,-0.103696,0.285108,0.175839,0.570441,0.244988,-0.800344,0.444726,-0.333629,0.123883,-0.103696,0.285108,0.175839,0.570441,0.244988,-0.800344,0.444726,-0.333629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6569,0.432194,0.823649,0.622710,0.765114,1.144031,0.905843,-0.011446,1.140681,-0.333629,-0.157225,0.060833,0.327308,0.786507,-0.044120,0.102935,-0.011446,0.459200,-0.333629,-0.311380,-0.253267,-0.024361,0.438387,-0.085090,-0.168819,0.838135,0.210731,-0.333629,-0.510876,-0.417796,

In [8]:
#Permutation (target and others)
#add target in the end of the List
#9*C80*C81*C82*..*C88 == 9 * 256 ==2304
#targetList = ["Total Solids",	"SS", 	"BOD5",	"NH3",	"Org-N",	"P-TOT",	"SO4",	"TKN",	"PRCP_NOOA"]
targetList = ["Total Solids"]
#combination
def combinationAndTarget(targetList, minlag, maxlag, separator='_'):
    combinationList = []
    combinationListExtend = []
    for target in targetList:
        one = [target]
        combinationList.append(one)
        temp = targetList[:]
        restList = temp.remove(target)
        for i in range(1, len(temp)+1):
          itert = itertools.combinations(temp, i)
          for t in itert:
            t = list(t)
            t.append(target)
            combinationList.append(t)
    
    #print(len(combinationList))
    #target
    targetList = []
    for combination in combinationList:
        tempList = []
        for i in range(minlag, maxlag+1): 
            for c in combination:
                tempList.append(c + separator + str(i))
        combinationListExtend.append(tempList)
        targetList.append(combination[-1])
    return combinationList, combinationListExtend, targetList

combinationList, combinationListExtend, targetList = combinationAndTarget(targetList, 1, lag)
#print(combinationList)
print(targetList)
#print(len(combinationList))
#print(len(combinationList) == len(targetList))
#X1 = X[str(name) for name in combinationList[3]]
#X[combinationList[3]]

['Total Solids']


In [9]:
#test
#X lag: data, combination: using features, tragetname: targetname, num: sequence number, series_narmalized: original normalized data
#lag: delay date, num_features: number of features len of combination
def RNN_LSTM(X, combination, targetname, series_normalized, lag):
    if len(combination) % lag == 0:
        num_features = int(len(combination) / lag)
    else:
        return "False"
    targetDF = series_normalized[targetname].to_frame()
    #print(X_test)
    X = X[combination]
    size = int(len(X) * 0.8) #split the data into training set (80%) and validation set
    train = X[0:size]
    train_target = targetDF[0:size]
    test = X[size:len(X)]
    test_target = targetDF[size:len(X)]
    X_train = train.values.reshape(-1, lag, num_features).astype('float32')
    y_train = train_target.values.astype('float32')

    X_test = test.values.reshape(-1, lag, num_features).astype('float32')
    y_test = test_target.values.astype('float32')

    hidden = lag * num_features + 1
    batch_size = 20
    epochs = 25

    model = Sequential()
    model.add(Bidirectional(LSTM(hidden), input_shape=(lag, num_features)))
    model.add(Dense(1))

    model.compile(optimizer='adam',
                loss='mae',
                metrics=['mae'])

    history = model.fit(X_train, y_train,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_data=(X_test,y_test))
    model.save('my_models/modelsep' + str(targetname))
    val = np.array(history.history['mae'])
    return val.mean()

In [10]:
# begin = 0
# #with open("combinationResult.csv", 'w') as fw:
# while(1):
#     with open("combinationResult.csv", 'a') as fw:
#         #for i in range(begin, 4):
#         for i in range(begin, len(targetList)+1):
#             maeResult = RNN_LSTM(X, combinationListExtend[i], targetList[i], series_normalized, lag)
#             fw.writelines([str(i)+',' + str(maeResult) + ',' + str(targetList[i]) + ',' + (str(combinationList[i])).replace(',','|')+ '\n'])
#             if i % 30 == 0:
#                 fw.flush()
#                 begin = i + 1
#                 break
#     if i >= len(targetList) or begin == len(targetList)+1:
#         print("done")
#         break
        


In [11]:
#one point input 
def extendList(inputList, lag, separator='_'):
    combinationListExtend = []
    for combination in inputList:
        tempList = []
        for i in range(1, lag+1): 
            for c in combination:
                tempList.append(c + separator + str(i))
        combinationListExtend.append(tempList)
    return combinationListExtend[0]
inputList = [["Total Solids",	"SS", 	"BOD5",	"NH3",	"Org-N",	"P-TOT",	"SO4",	"TKN",	"PRCP_NOOA"]]
inputListExtend = extendList(inputList, 7)
print(inputListExtend[0])
inputTarget = "Total Solids"
maeResult = RNN_LSTM(X, inputListExtend, inputTarget, series_normalized, lag)
print(maeResult)

Total Solids_1
Epoch 1/25
263/263 [==============================] - 3s 10ms/step - loss: 0.5363 - mae: 0.5363 - val_loss: 0.4524 - val_mae: 0.4524
Epoch 2/25
263/263 [==============================] - 2s 7ms/step - loss: 0.5038 - mae: 0.5038 - val_loss: 0.4563 - val_mae: 0.4563
Epoch 3/25
263/263 [==============================] - 2s 7ms/step - loss: 0.4917 - mae: 0.4917 - val_loss: 0.4500 - val_mae: 0.4500
Epoch 4/25
263/263 [==============================] - 2s 7ms/step - loss: 0.4851 - mae: 0.4851 - val_loss: 0.4488 - val_mae: 0.4488
Epoch 5/25
263/263 [==============================] - 2s 7ms/step - loss: 0.4792 - mae: 0.4792 - val_loss: 0.4501 - val_mae: 0.4501
Epoch 6/25
263/263 [==============================] - 2s 7ms/step - loss: 0.4756 - mae: 0.4756 - val_loss: 0.4468 - val_mae: 0.4468
Epoch 7/25
263/263 [==============================] - 2s 7ms/step - loss: 0.4734 - mae: 0.4734 - val_loss: 0.4409 - val_mae: 0.4409
Epoch 8/25
263/263 [==============================] - 2s 7ms

In [54]:
#input start and end date(M/D/Y), target
#output prediction, real value, error.
import pandas as pd
output_targets = ["Total Solids"]
start_date = "01/01/2018"
end_date = "01/01/2019"

timelist = pd.date_range('1/1/2001','12/31/2018')
input_datasetdf = pd.DataFrame(X.values, index=timelist)[start_date: end_date]
input_dataset = input_datasetdf.values.reshape(-1, lag, num_features).astype('float32')
#Error mae mape
def mae_mape(actual, pred):
    actual, pred = np.array(actual), np.array(pred)
    return np.abs(actual - pred), np.mean(np.abs(actual - pred)), np.mean(np.abs(actual - pred) / actual)

#return a dictionary
#list predict values,list actual values, array error, mae, mape no %
def predictWithModel(start_date=start_date, end_date=end_date, output_targets=output_targets, input_dataset=input_dataset, realdata=series, meandata=meandata, stddata=stddata):
    ansdict = {}
    for target in output_targets: 
        model_using = tf.keras.models.load_model('my_models/modelsep' + str(target))
        y_predict = model_using.predict(input_dataset)
        origin_predict = []
        for ans in y_predict:
            origin_predict.append(ans[0] * stddata[target_dict[str(target)]] +  meandata[target_dict[str(target)]])
        #print(origin_predict)
        print(list(realdata.loc[start_date: end_date][str(target)].values))
        error, mae, mape= mae_mape(origin_predict, list(realdata.loc[start_date: end_date][str(target)].values))
        ansdict[str(target)] = (origin_predict, list(realdata.loc[start_date: end_date][str(target)].values), error, mae, mape)
    return ansdict
t = predictWithModel()
print(t)

[704.0, 738.0, 814.0, 830.0, 810.0, 824.0, 764.0, 902.0, 1044.0, 982.0, 1400.0, 1282.0, 970.0, 784.0, 910.0, 1176.0, 1180.0, 1002.0, 884.0, 996.0, 968.0, 1237.0, 1430.0, 1204.0, 1015.0, 1007.0, 982.0, 988.0, 984.0, 1003.0, 934.0, 908.0, 874.0, 834.0, 866.0, 944.0, 1284.0, 1303.0, 1040.0, 1086.0, 1106.0, 1075.0, 1098.0, 1216.0, 1322.0, 1765.0, 1597.0, 1404.0, 1412.0, 1709.0, 882.0, 886.0, 902.0, 920.0, 862.0, 992.0, 1013.0, 1020.0, 1248.0, 800.0, 922.0, 954.0, 960.0, 1010.0, 1182.0, 1560.0, 1040.0, 1178.0, 1046.0, 1018.0, 1005.0, 1032.0, 1030.0, 974.0, 984.0, 966.0, 914.0, 946.0, 972.0, 1036.0, 1003.0, 994.0, 1011.0, 1072.0, 948.0, 1060.0, 976.0, 1026.0, 1046.0, 980.0, 940.0, 972.0, 980.0, 968.0, 1033.0, 1074.0, 1177.0, 870.0, 1720.0, 1012.0, 1054.0, 1134.0, 1046.0, 882.0, 638.0, 790.0, 944.0, 1207.0, 1056.0, 1042.0, 1080.0, 984.0, 1056.0, 1046.0, 1042.0, 1004.0, 998.0, 994.0, 936.0, 1016.0, 1071.0, 928.0, 574.0, 594.0, 540.0, 794.0, 1083.0, 998.0, 1738.0, 962.0, 1028.0, 1046.0, 736.0, 

In [51]:
series[start_date: end_date]

AssertionError: ignored

pandas.core.indexes.datetimes.DatetimeIndex